# Sensitivity Analysis and Monte Carlo Stress Testing

This notebook performs comprehensive parameter sensitivity analysis using Monte Carlo methods.

## Objectives

1. Run Monte Carlo parameter sweeps
2. Generate collapse probability heatmaps
3. Identify structural failure regions
4. Quantify parameter sensitivities
5. Document key findings about collapse conditions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import sys
sys.path.insert(0, '..')

from model import SystemParameters
from experiments import (
    run_collateral_shock_experiment,
    run_monte_carlo_stress_test,
    MonteCarloResults
)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 12

# Set seed for reproducibility
np.random.seed(42)

## 1. Single Parameter Sensitivity

First, we analyze how individual parameters affect collapse probability.

In [ ]:
def run_sensitivity_test(param_name, param_range, n_trials=20, shock_mag=-0.3):
    """Test sensitivity to a single parameter."""
    results = []
    
    for val in param_range:
        collapses = 0
        peg_devs = []
        drawdowns = []
        
        for trial in range(n_trials):
            params = SystemParameters(random_seed=42 + trial)
            setattr(params, param_name, val)
            
            # Randomize shock timing slightly
            shock_time = np.random.randint(80, 120)
            
            result = run_collateral_shock_experiment(
                shock_magnitude=shock_mag,
                shock_time=shock_time,
                params=params,
                n_steps=500
            )
            
            if result.time_to_collapse < np.inf:
                collapses += 1
            peg_devs.append(result.peg_deviation_integral)
            drawdowns.append(result.max_drawdown)
        
        results.append({
            'value': val,
            'collapse_prob': collapses / n_trials,
            'mean_peg_dev': np.mean(peg_devs),
            'std_peg_dev': np.std(peg_devs),
            'mean_drawdown': np.mean(drawdowns),
            'std_drawdown': np.std(drawdowns)
        })
    
    return results

# Test liquidity depth sensitivity
print("Testing liquidity_depth sensitivity...")
liq_range = np.linspace(1e5, 2e6, 10)
liq_results = run_sensitivity_test('liquidity_depth', liq_range, n_trials=30)

# Test mint coefficient sensitivity  
print("Testing mint_coefficient sensitivity...")
mint_range = np.linspace(0.05, 0.5, 10)
mint_results = run_sensitivity_test('mint_coefficient', mint_range, n_trials=30)

# Test demand elasticity sensitivity
print("Testing demand_elasticity sensitivity...")
demand_range = np.linspace(0.1, 1.0, 10)
demand_results = run_sensitivity_test('demand_elasticity', demand_range, n_trials=30)

print("Sensitivity tests complete!")

In [ ]:
# Plot single parameter sensitivities
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Liquidity depth
vals = [r['value']/1e6 for r in liq_results]
probs = [r['collapse_prob'] for r in liq_results]
axes[0, 0].plot(vals, probs, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Liquidity Depth (millions)')
axes[0, 0].set_ylabel('Collapse Probability')
axes[0, 0].set_title('Collapse Probability vs Liquidity Depth')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_ylim([0, 1])

devs = [r['mean_peg_dev'] for r in liq_results]
stds = [r['std_peg_dev'] for r in liq_results]
axes[1, 0].errorbar(vals, devs, yerr=stds, fmt='ro-', linewidth=2, markersize=8, capsize=3)
axes[1, 0].set_xlabel('Liquidity Depth (millions)')
axes[1, 0].set_ylabel('Peg Deviation Integral')
axes[1, 0].set_title('Peg Deviation vs Liquidity Depth')
axes[1, 0].grid(True, alpha=0.3)

# Mint coefficient
vals = [r['value'] for r in mint_results]
probs = [r['collapse_prob'] for r in mint_results]
axes[0, 1].plot(vals, probs, 'go-', linewidth=2, markersize=8)
axes[0, 1].set_xlabel('Mint Coefficient')
axes[0, 1].set_ylabel('Collapse Probability')
axes[0, 1].set_title('Collapse Probability vs Mint Coefficient')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim([0, 1])

devs = [r['mean_peg_dev'] for r in mint_results]
stds = [r['std_peg_dev'] for r in mint_results]
axes[1, 1].errorbar(vals, devs, yerr=stds, fmt='mo-', linewidth=2, markersize=8, capsize=3)
axes[1, 1].set_xlabel('Mint Coefficient')
axes[1, 1].set_ylabel('Peg Deviation Integral')
axes[1, 1].set_title('Peg Deviation vs Mint Coefficient')
axes[1, 1].grid(True, alpha=0.3)

# Demand elasticity
vals = [r['value'] for r in demand_results]
probs = [r['collapse_prob'] for r in demand_results]
axes[0, 2].plot(vals, probs, 'co-', linewidth=2, markersize=8)
axes[0, 2].set_xlabel('Demand Elasticity')
axes[0, 2].set_ylabel('Collapse Probability')
axes[0, 2].set_title('Collapse Probability vs Demand Elasticity')
axes[0, 2].grid(True, alpha=0.3)
axes[0, 2].set_ylim([0, 1])

devs = [r['mean_peg_dev'] for r in demand_results]
stds = [r['std_peg_dev'] for r in demand_results]
axes[1, 2].errorbar(vals, devs, yerr=stds, fmt='yo-', linewidth=2, markersize=8, capsize=3)
axes[1, 2].set_xlabel('Demand Elasticity')
axes[1, 2].set_ylabel('Peg Deviation Integral')
axes[1, 2].set_title('Peg Deviation vs Demand Elasticity')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/plots/parameter_sensitivity.png', dpi=300, bbox_inches='tight')
plt.show()

## 2. Monte Carlo Parameter Sweep

Run comprehensive Monte Carlo stress tests across multiple parameter combinations.

In [ ]:
# Run Monte Carlo stress test
print("Running Monte Carlo stress test...")
print("This may take a few minutes...")

base_params = SystemParameters(random_seed=42)

# Test mint and burn coefficients
mc_results = run_monte_carlo_stress_test(
    param_ranges={
        'mint_coefficient': (0.05, 0.50),
        'burn_coefficient': (0.05, 0.50)
    },
    n_trials=20,  # Increase for more accurate results
    base_params=base_params,
    n_steps=500
)

print("Monte Carlo complete!")
print(f"Collapse probability matrix shape: {mc_results.collapse_probabilities.shape}")

In [ ]:
# Plot Monte Carlo heatmap
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

mint_vals = mc_results.parameter_grid['mint_coefficient']
burn_vals = mc_results.parameter_grid['burn_coefficient']

# Collapse probability heatmap
im1 = axes[0].imshow(mc_results.collapse_probabilities, aspect='auto', origin='lower',
                     extent=[mint_vals[0], mint_vals[-1], burn_vals[0], burn_vals[-1]],
                     cmap='RdYlGn_r', vmin=0, vmax=1)
axes[0].set_xlabel('Mint Coefficient')
axes[0].set_ylabel('Burn Coefficient')
axes[0].set_title('Collapse Probability')
plt.colorbar(im1, ax=axes[0])

# Mean time to collapse
# Replace inf with a large number for visualization
ttc_viz = np.where(mc_results.mean_time_to_collapse == 0, np.nan, mc_results.mean_time_to_collapse)
im2 = axes[1].imshow(ttc_viz, aspect='auto', origin='lower',
                     extent=[mint_vals[0], mint_vals[-1], burn_vals[0], burn_vals[-1]],
                     cmap='viridis')
axes[1].set_xlabel('Mint Coefficient')
axes[1].set_ylabel('Burn Coefficient')
axes[1].set_title('Mean Time to Collapse')
plt.colorbar(im2, ax=axes[1])

# Mean max drawdown
im3 = axes[2].imshow(mc_results.mean_max_drawdown, aspect='auto', origin='lower',
                     extent=[mint_vals[0], mint_vals[-1], burn_vals[0], burn_vals[-1]],
                     cmap='hot')
axes[2].set_xlabel('Mint Coefficient')
axes[2].set_ylabel('Burn Coefficient')
axes[2].set_title('Mean Max Drawdown')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.savefig('../results/plots/monte_carlo_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Liquidity Threshold Analysis

Identify the critical liquidity threshold below which collapse is highly likely.

In [ ]:
# Detailed liquidity threshold analysis
print("Running liquidity threshold analysis...")

liquidity_ratios = np.linspace(0.1, 1.0, 20)  # As fraction of initial supply
n_trials = 50
shock_mags = [-0.2, -0.3, -0.4, -0.5]

results_by_shock = {}

for shock in shock_mags:
    collapse_probs = []
    
    for ratio in liquidity_ratios:
        collapses = 0
        liq = ratio * 1e6  # ratio * initial_supply
        
        for trial in range(n_trials):
            params = SystemParameters(
                initial_liquidity=liq,
                liquidity_depth=liq,
                random_seed=42 + trial
            )
            
            shock_time = np.random.randint(80, 120)
            result = run_collateral_shock_experiment(
                shock_magnitude=shock,
                shock_time=shock_time,
                params=params,
                n_steps=500
            )
            
            if result.time_to_collapse < np.inf:
                collapses += 1
        
        collapse_probs.append(collapses / n_trials)
    
    results_by_shock[shock] = collapse_probs

print("Analysis complete!")

In [ ]:
# Plot liquidity threshold results
fig, ax = plt.subplots(figsize=(12, 8))

colors = ['blue', 'green', 'orange', 'red']
for i, (shock, probs) in enumerate(results_by_shock.items()):
    ax.plot(liquidity_ratios, probs, 'o-', color=colors[i], 
            linewidth=2, markersize=8, label=f'Shock: {shock:.0%}')

# Mark 50% collapse probability line
ax.axhline(y=0.5, color='gray', linestyle='--', linewidth=2, label='50% collapse prob')

# Mark critical threshold region
ax.axvspan(0.1, 0.3, alpha=0.2, color='red', label='Critical region')

ax.set_xlabel('Liquidity / Initial Supply Ratio', fontsize=12)
ax.set_ylabel('Collapse Probability', fontsize=12)
ax.set_title('Collapse Probability vs Liquidity Ratio\n(Different Shock Magnitudes)', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 1.05])

plt.tight_layout()
plt.savefig('../results/plots/liquidity_threshold.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Statistical Analysis and Confidence Intervals

In [ ]:
# Compute confidence intervals for collapse probability
def compute_ci(successes, trials, confidence=0.95):
    """Compute Wilson score confidence interval."""
    z = stats.norm.ppf((1 + confidence) / 2)
    p = successes / trials
    
    denominator = 1 + z**2 / trials
    center = (p + z**2 / (2 * trials)) / denominator
    margin = z * np.sqrt((p * (1 - p) + z**2 / (4 * trials)) / trials) / denominator
    
    return max(0, center - margin), min(1, center + margin)

# Run larger trial for final statistics
print("Running final statistical analysis...")

# Test low liquidity scenario
n_final_trials = 100
low_liq_params = SystemParameters(
    initial_liquidity=3e5,  # 0.3 × initial_supply
    liquidity_depth=3e5
)

collapses = 0
time_to_collapses = []
max_drawdowns = []

for trial in range(n_final_trials):
    low_liq_params.random_seed = 42 + trial
    shock_mag = np.random.uniform(-0.5, -0.2)
    shock_time = np.random.randint(50, 150)
    
    result = run_collateral_shock_experiment(
        shock_magnitude=shock_mag,
        shock_time=shock_time,
        params=low_liq_params,
        n_steps=500
    )
    
    if result.time_to_collapse < np.inf:
        collapses += 1
        time_to_collapses.append(result.time_to_collapse)
    max_drawdowns.append(result.max_drawdown)

collapse_prob = collapses / n_final_trials
ci_low, ci_high = compute_ci(collapses, n_final_trials)

print("\n" + "=" * 70)
print("KEY FINDING: STRUCTURAL COLLAPSE UNDER LIQUIDITY CONSTRAINTS")
print("=" * 70)
print(f"\nCondition: Liquidity depth < 0.3 × initial_supply")
print(f"Shock distribution: Uniform[-50%, -20%]")
print(f"Monte Carlo trials: n = {n_final_trials}")
print(f"\nResults:")
print(f"  Collapse probability: {collapse_prob:.1%} (95% CI: {ci_low:.1%}-{ci_high:.1%})")
if time_to_collapses:
    print(f"  Mean time to collapse: {np.mean(time_to_collapses):.1f} ± {np.std(time_to_collapses):.1f} time steps")
print(f"  Mean max drawdown: {np.mean(max_drawdowns):.4f} ± {np.std(max_drawdowns):.4f}")
print(f"\nConclusion: Systems with liquidity below 30% of initial supply")
print(f"exhibit structural vulnerability to collapse under moderate stress.")

## 5. Summary of Findings

In [ ]:
# Save summary statistics to CSV
import csv

summary_data = [
    ['Metric', 'Value', 'CI_Low', 'CI_High', 'Notes'],
    ['Collapse_Probability_Low_Liquidity', f'{collapse_prob:.4f}', f'{ci_low:.4f}', f'{ci_high:.4f}', 'L < 0.3*S0'],
    ['Mean_Time_to_Collapse', f'{np.mean(time_to_collapses):.2f}' if time_to_collapses else 'N/A', 
     f'{np.mean(time_to_collapses) - np.std(time_to_collapses):.2f}' if time_to_collapses else 'N/A',
     f'{np.mean(time_to_collapses) + np.std(time_to_collapses):.2f}' if time_to_collapses else 'N/A',
     'Time steps'],
    ['Mean_Max_Drawdown', f'{np.mean(max_drawdowns):.4f}', 
     f'{np.mean(max_drawdowns) - np.std(max_drawdowns):.4f}',
     f'{np.mean(max_drawdowns) + np.std(max_drawdowns):.4f}',
     'Price drop from 1.0'],
    ['N_Trials', str(n_final_trials), '', '', 'Monte Carlo samples']
]

with open('../results/data/sensitivity_summary.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(summary_data)

print("Summary saved to results/data/sensitivity_summary.csv")

# Display final summary
print("\n" + "=" * 70)
print("SENSITIVITY ANALYSIS COMPLETE")
print("=" * 70)
print("\nGenerated outputs:")
print("  - parameter_sensitivity.png: Single parameter sensitivity curves")
print("  - monte_carlo_heatmap.png: 2D parameter sweep results")
print("  - liquidity_threshold.png: Liquidity threshold analysis")
print("  - sensitivity_summary.csv: Numerical results")
print("\nKey structural finding:")
print(f"  Systems with liquidity < 30% of initial supply have")
print(f"  {collapse_prob:.1%} collapse probability under realistic stress.")